In [ ]:
import ipywidgets as widgets
from IPython.display import display

def assume_direction(kite, instruments, lp, up):
    """
    Displays Bullish/Bearish strategy buttons, runs the selected strategy,
    and collects futures and options symbols.

    Parameters:
    - kite: Kite Connect object
    - instruments: list of instruments
    - lp: lower premium
    - up: upper premium

    Returns:
    - fut_symbols: list of selected futures symbols (global)
    - option_symbols: list of selected option symbols (global)
    """

    # Output widget to display messages
    output = widgets.Output()

    # Initialize global variables
    global fut_symbols, option_symbols
    fut_symbols = []
    option_symbols = []

    # Function to handle bullish button click
    def on_bullish_click(b):
        global fut_symbols, option_symbols
        (fut_symbol, fut_token), (put_symbol, put_token) = bull_put_spread_with_future(
            kite, instruments, lower_premium=lp, upper_premium=up
        )
        fut_symbols = [fut_symbol]
        option_symbols = [put_symbol]

        with output:
            output.clear_output()
            print("Bullish strategy selected ✅")
            print("Collected Futures:", fut_symbols)
            print("Collected Options:", option_symbols)

    # Function to handle bearish button click
    def on_bearish_click(b):
        global fut_symbols, option_symbols
        (fut_symbol, fut_token), (call_symbol, call_token) = bear_call_spread_with_future(
            kite, instruments, lower_premium=lp, upper_premium=up
        )
        fut_symbols = [fut_symbol]
        option_symbols = [call_symbol]

        with output:
            output.clear_output()
            print("Bearish strategy selected ✅")
            print("Collected Futures:", fut_symbols)
            print("Collected Options:", option_symbols)

    # Create buttons
    bullish_button = widgets.Button(description="Bullish Strategy", button_style='success')
    bearish_button = widgets.Button(description="Bearish Strategy", button_style='danger')

    # Connect buttons to functions
    bullish_button.on_click(on_bullish_click)
    bearish_button.on_click(on_bearish_click)

    # Display buttons and output widget
    display(widgets.VBox([widgets.HBox([bullish_button, bearish_button]), output]))

    # Return global variables (they will update when buttons are clicked)
    return fut_symbols, option_symbols


In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

def make_order_basket(fut_symbols, option_symbols, kite):
    orders = []
    qty_options = [75, 150, 225, 300, 375, 450]
    kite_orange = '#f57c00'

    # -----------------------------
    # Output Widgets
    # -----------------------------
    basket_output = widgets.Output()   # Shows order basket
    msg_output = widgets.Output()      # Shows margin info, placement messages

    # -----------------------------
    # Quantity Buttons
    # -----------------------------
    def create_qty_buttons():
        selected_qty = qty_options[0]
        buttons = []
        for q in qty_options:
            btn = widgets.Button(description=str(q),
                                 layout=widgets.Layout(width='60px', margin='0 8px 0 0'))
            buttons.append(btn)

        qty_out = widgets.Output()
        qty_box = widgets.VBox([widgets.HBox(buttons + [qty_out])],
                               layout=widgets.Layout(margin='0 0 10px 0'))

        def update_styles(clicked_btn):
            nonlocal selected_qty
            selected_qty = int(clicked_btn.description)
            for b in buttons:
                b.style.button_color = kite_orange if b == clicked_btn else None
            with qty_out:
                clear_output()

        for b in buttons:
            b.on_click(update_styles)

        update_styles(buttons[0])
        return qty_box, lambda: selected_qty

    # -----------------------------
    # Buy/Sell Buttons
    # -----------------------------
    def create_side_buttons(default='BUY'):
        selected_side = default
        buy_btn = widgets.Button(description="BUY",
                                 layout=widgets.Layout(width='60px', margin='0 8px 0 0'))
        sell_btn = widgets.Button(description="SELL",
                                  layout=widgets.Layout(width='60px', margin='0 8px 0 0'))

        side_box = widgets.VBox([widgets.HBox([buy_btn, sell_btn])],
                                layout=widgets.Layout(margin='0 0 10px 0'))

        def update_styles(btn_clicked):
            nonlocal selected_side
            selected_side = btn_clicked.description
            buy_btn.style.button_color = '#2196f3' if selected_side == 'BUY' else '#ccc'
            sell_btn.style.button_color = 'red' if selected_side == 'SELL' else '#ccc'

        buy_btn.on_click(update_styles)
        sell_btn.on_click(update_styles)
        update_styles(widgets.Button(description=selected_side))
        return side_box, lambda: selected_side

    # -----------------------------
    # Order Widget Set
    # -----------------------------
    def create_order_widget(symbols_list, default_buy=True):
        tradingsymbol = widgets.Dropdown(options=symbols_list, description="Symbol:",
                                         layout=widgets.Layout(margin='0 0 10px 0'))
        side_box, get_side = create_side_buttons('BUY' if default_buy else 'SELL')
        qty_box, get_qty = create_qty_buttons()
        order_type = widgets.Dropdown(options=["MARKET", "LIMIT","SL", "SL-M"], value="MARKET",
                                      description="Type:", layout=widgets.Layout(margin='0 0 10px 0'))
        product = widgets.Dropdown(options=["MIS", "CNC", "NRML"], value="MIS",
                                   description="Product:", layout=widgets.Layout(margin='0 0 10px 0'))
        variety = widgets.Dropdown(options=["regular", "amo"], value="regular",
                                   description="Variety:", layout=widgets.Layout(margin='0 0 10px 0'))
        add_btn = widgets.Button(description="Add Order", button_style="primary",
                                 layout=widgets.Layout(margin='0 0 20px 0'))

        return {
            "tradingsymbol": tradingsymbol,
            "get_side": get_side,
            "side_box": side_box,
            "qty_box": qty_box,
            "get_qty": get_qty,
            "order_type": order_type,
            "product": product,
            "variety": variety,
            "add_btn": add_btn
        }

    fut_widget = create_order_widget(fut_symbols, default_buy=True)
    opt_widget = create_order_widget(option_symbols, default_buy=True)

    # -----------------------------
    # Add Order Logic
    # -----------------------------
    def add_order_widget(widget):
        sym = widget["tradingsymbol"].value
        txn_type = widget["get_side"]()
        qty = widget["get_qty"]()

        if sym.endswith(("CE", "PE")):
            txn_type = "BUY"

        order = {
            "tradingsymbol": sym,
            "exchange": "NFO",
            "transaction_type": txn_type,
            "quantity": qty,
            "order_type": widget["order_type"].value,
            "product": widget["product"].value,
            "variety": widget["variety"].value
        }

        orders.append(order)

        with basket_output:
            clear_output()
            display(HTML("<span style='color: orange; font-weight: bold;'>🛒 Order Basket:</span>"))
            for o in orders:
                side_color = "green" if o["transaction_type"] == "BUY" else "red"
                box_style = (
                    "display: inline-block; padding: 5px 10px; margin: 2px 4px; "
                    "border-radius: 5px; font-family: monospace; font-weight: bold; color: white;"
                )
                symbol_box = f"<span style='{box_style} background-color: #2196f3;'>{o['tradingsymbol']}</span>"
                qty_box = f"<span style='{box_style} background-color: #2196f3;'>{o['quantity']}</span>"
                side_box_html = f"<span style='{box_style} background-color: {side_color};'>{o['transaction_type']}</span>"
                type_box = f"<span style='{box_style} background-color: #2196f3;'>{o['order_type']}</span>"
                product_box = f"<span style='{box_style} background-color: #2196f3;'>{o['product']}</span>"
                variety_box = f"<span style='{box_style} background-color: #2196f3;'>{o['variety']}</span>"
                display(HTML(symbol_box + qty_box + side_box_html + type_box + product_box + variety_box))

    fut_widget["add_btn"].on_click(lambda b: add_order_widget(fut_widget))
    opt_widget["add_btn"].on_click(lambda b: add_order_widget(opt_widget))

    # -----------------------------
    # Clear Orders
    # -----------------------------
    clear_btn = widgets.Button(description="Clear Orders", button_style="danger")
    def clear_orders(b):
        orders.clear()
        with basket_output:
            clear_output()
            print("🗑️ All orders cleared.")
        with msg_output:
            clear_output()
    clear_btn.on_click(clear_orders)

    # -----------------------------
    # Check Margin
    # -----------------------------
    margin_btn = widgets.Button(description="Check Margin", button_style="info")
    def check_margin_required():
        if not orders:
            with msg_output:
                clear_output()
                print("⚠️ No orders to check margin for!")
            return None, None

        margin_orders = []
        for o in orders:
            # ex = "NSE" if not o["tradingsymbol"].endswith(("CE","PE")) else "NFO"
            ex = "NFO"
            margin_orders.append({
                "exchange": ex,
                "tradingsymbol": o["tradingsymbol"],
                "transaction_type": o["transaction_type"],
                "quantity": o["quantity"],
                "product": o["product"],
                "order_type": o["order_type"]
            })

        try:
            margins = kite.margins()
            margin_info = kite.basket_order_margins(margin_orders, consider_positions=True, mode="compact")
            available_balance = margins['equity']['net']
            df = pd.json_normalize(margin_info['orders'], sep='_')
            total_required = df['total'].sum()

            with msg_output:
                clear_output()
                print(f"💰 Available Balance: ₹{available_balance:,.2f}")
                print(f"📊 Margin Required: ₹{total_required:,.2f}")
                if total_required > available_balance:
                    print("⚠️ Insufficient funds to place these orders!")
                else:
                    print("✅ Sufficient funds available.")

            return total_required, available_balance
        except Exception as e:
            with msg_output:
                clear_output()
                print(f"❌ Error checking margin: {e}")
            return None, None

    margin_btn.on_click(lambda b: check_margin_required())

    # -----------------------------
    # Internal function to execute orders
    # -----------------------------
    def _execute_orders():
        with msg_output:
            clear_output()
            print("🚀 Placing all orders...\n")

        for order in orders:
            try:
                order_id = kite.place_order(
                    variety=order["variety"],
                    exchange=order["exchange"],
                    tradingsymbol=order["tradingsymbol"],
                    transaction_type=order["transaction_type"],
                    quantity=order["quantity"],
                    order_type=order["order_type"],
                    product=order["product"]
                )
                with msg_output:
                    display(HTML(f"<pre style='color: green; font-weight: bold'>✔ Order placed: {order['tradingsymbol']} → Order ID: {order_id}</pre>"))
            except Exception as e:
                with msg_output:
                    display(HTML(f"<pre style='color: red; font-weight: bold'>❌ Error placing {order['tradingsymbol']}: {e}</pre>"))

    # -----------------------------
    # Place Orders with insufficient funds confirmation
    # -----------------------------
    place_btn = widgets.Button(description="Place All Orders", button_style="success")

    def place_orders(b):
        if not orders:
            with msg_output:
                clear_output()
                print("⚠️ No orders to place!")
            return

        total_required, available_balance = check_margin_required()
        if total_required is None:
            return

        if total_required > available_balance:
            # Show confirmation Yes/No
            yes_btn = widgets.Button(description="Yes", button_style="success")
            no_btn = widgets.Button(description="No", button_style="danger")
            confirm_box = widgets.HBox([yes_btn, no_btn])

            with msg_output:
                print(f"⚠️ Insufficient funds! Required: ₹{total_required:,.2f}, Available: ₹{available_balance:,.2f}")
                print("Do you still want to proceed with placing orders?")
                display(confirm_box)

            def proceed_yes(b):
                confirm_box.close()
                _execute_orders()

            def proceed_no(b):
                with msg_output:
                    clear_output()
                    print("❌ Order placement cancelled.")

            yes_btn.on_click(proceed_yes)
            no_btn.on_click(proceed_no)
        else:
            _execute_orders()

    place_btn.on_click(place_orders)

    # -----------------------------
    # Layout FUTURES and OPTIONS
    # -----------------------------
    fut_box = widgets.VBox([
        widgets.HTML("<h4>FUTURES</h4>"),
        fut_widget["tradingsymbol"],
        fut_widget["side_box"],
        fut_widget["qty_box"],
        fut_widget["order_type"],
        fut_widget["product"],
        fut_widget["variety"],
        fut_widget["add_btn"]
    ], layout=widgets.Layout(min_width='320px'))

    opt_box = widgets.VBox([
        widgets.HTML("<h4>OPTIONS</h4>"),
        opt_widget["tradingsymbol"],
        opt_widget["side_box"],
        opt_widget["qty_box"],
        opt_widget["order_type"],
        opt_widget["product"],
        opt_widget["variety"],
        opt_widget["add_btn"]
    ], layout=widgets.Layout(min_width='320px'))

    container = widgets.HBox(
        [fut_box, opt_box],
        layout=widgets.Layout(
            justify_content='space-between',
            width='100%',
            padding='0 50px'
        )
    )

    # -----------------------------
    # Display everything
    # -----------------------------
    display(container)
    display(widgets.HBox([place_btn, clear_btn, margin_btn], layout=widgets.Layout(margin='20px 0 0 0')))
    display(basket_output)
    display(msg_output)


In [ ]:
# import ipywidgets as widgets
# from IPython.display import display, clear_output, HTML

# def make_order_basket(fut_symbols, option_symbols):
#     orders = []
#     qty_options = [75, 150, 225, 300, 375, 450]
#     kite_orange = '#f57c00'
#     output = widgets.Output()
    
#     # -----------------------------
#     # Quantity Buttons
#     # -----------------------------
#     def create_qty_buttons():
#         selected_qty = qty_options[0]
#         buttons = []
#         for q in qty_options:
#             btn = widgets.Button(description=str(q),
#                                  layout=widgets.Layout(width='60px', margin='0 8px 0 0'))
#             buttons.append(btn)

#         qty_out = widgets.Output()
#         qty_box = widgets.VBox([widgets.HBox(buttons + [qty_out])],
#                                layout=widgets.Layout(margin='0 0 10px 0'))  # spacing after qty buttons

#         def update_styles(clicked_btn):
#             nonlocal selected_qty
#             selected_qty = int(clicked_btn.description)
#             for b in buttons:
#                 b.style.button_color = kite_orange if b == clicked_btn else None
#             with qty_out:
#                 clear_output()

#         for b in buttons:
#             b.on_click(update_styles)

#         update_styles(buttons[0])
#         return qty_box, lambda: selected_qty

#     # -----------------------------
#     # Buy/Sell Buttons
#     # -----------------------------
#     def create_side_buttons(default='BUY'):
#         selected_side = default
#         buy_btn = widgets.Button(description="BUY",
#                                  layout=widgets.Layout(width='60px', margin='0 8px 0 0'))
#         sell_btn = widgets.Button(description="SELL",
#                                   layout=widgets.Layout(width='60px', margin='0 8px 0 0'))

#         side_box = widgets.VBox([widgets.HBox([buy_btn, sell_btn])],
#                                 layout=widgets.Layout(margin='0 0 10px 0'))  # spacing after side buttons

#         def update_styles(btn_clicked):
#             nonlocal selected_side
#             selected_side = btn_clicked.description
#             buy_btn.style.button_color = '#2196f3' if selected_side == 'BUY' else '#ccc'
#             sell_btn.style.button_color = 'red' if selected_side == 'SELL' else '#ccc'

#         buy_btn.on_click(update_styles)
#         sell_btn.on_click(update_styles)
#         update_styles(widgets.Button(description=selected_side))
#         return side_box, lambda: selected_side

#     # -----------------------------
#     # Order Widget Set
#     # -----------------------------
#     def create_order_widget(symbols_list, default_buy=True):
#         tradingsymbol = widgets.Dropdown(options=symbols_list, description="Symbol:",
#                                          layout=widgets.Layout(margin='0 0 10px 0'))  # spacing after symbol
#         side_box, get_side = create_side_buttons('BUY' if default_buy else 'SELL')
#         qty_box, get_qty = create_qty_buttons()
#         order_type = widgets.Dropdown(options=["MARKET", "LIMIT"], value="MARKET",
#                                       description="Type:", layout=widgets.Layout(margin='0 0 10px 0'))
#         product = widgets.Dropdown(options=["MIS", "CNC", "NRML"], value="MIS",
#                                    description="Product:", layout=widgets.Layout(margin='0 0 10px 0'))
#         variety = widgets.Dropdown(options=["regular", "amo"], value="amo",
#                                    description="Variety:", layout=widgets.Layout(margin='0 0 10px 0'))
#         add_btn = widgets.Button(description="Add Order", button_style="primary",
#                                  layout=widgets.Layout(margin='0 0 20px 0'))

#         return {
#             "tradingsymbol": tradingsymbol,
#             "get_side": get_side,
#             "side_box": side_box,
#             "qty_box": qty_box,
#             "get_qty": get_qty,
#             "order_type": order_type,
#             "product": product,
#             "variety": variety,
#             "add_btn": add_btn
#         }

#     fut_widget = create_order_widget(fut_symbols, default_buy=True)
#     opt_widget = create_order_widget(option_symbols, default_buy=True)

#     # -----------------------------
#     # Add Order Logic
#     # -----------------------------
#     def add_order_widget(widget):
#         sym = widget["tradingsymbol"].value
#         txn_type = widget["get_side"]()
#         qty = widget["get_qty"]()

#         # If CE/PE, always BUY
#         if sym.endswith(("CE", "PE")):
#             txn_type = "BUY"

#         order = {
#             "tradingsymbol": sym,
#             "exchange": "NFO",
#             "transaction_type": txn_type,
#             "quantity": qty,
#             "order_type": widget["order_type"].value,
#             "product": widget["product"].value,
#             "variety": widget["variety"].value
#         }

#         orders.append(order)

#         with output:
#             clear_output()
#             display(HTML("<span style='color: orange; font-weight: bold;'>🛒 Order Basket:</span>"))
#             for o in orders:
#                 side_color = "green" if o["transaction_type"] == "BUY" else "red"
#                 box_style = (
#                     "display: inline-block; padding: 5px 10px; margin: 2px 4px; "
#                     "border-radius: 5px; font-family: monospace; font-weight: bold; color: white;"
#                 )
#                 symbol_box = f"<span style='{box_style} background-color: #2196f3;'>{o['tradingsymbol']}</span>"
#                 qty_box = f"<span style='{box_style} background-color: #2196f3;'>{o['quantity']}</span>"
#                 side_box_html = f"<span style='{box_style} background-color: {side_color};'>{o['transaction_type']}</span>"
#                 type_box = f"<span style='{box_style} background-color: #2196f3;'>{o['order_type']}</span>"
#                 product_box = f"<span style='{box_style} background-color: #2196f3;'>{o['product']}</span>"
#                 variety_box = f"<span style='{box_style} background-color: #2196f3;'>{o['variety']}</span>"
#                 display(HTML(symbol_box + qty_box + side_box_html + type_box + product_box + variety_box))

#     fut_widget["add_btn"].on_click(lambda b: add_order_widget(fut_widget))
#     opt_widget["add_btn"].on_click(lambda b: add_order_widget(opt_widget))

#     # -----------------------------
#     # Clear Orders
#     # -----------------------------
#     clear_btn = widgets.Button(description="Clear Orders", button_style="danger")
#     def clear_orders(b):
#         with output:
#             clear_output()
#             orders.clear()
#             print("🗑️ All orders cleared.")
#     clear_btn.on_click(clear_orders)

#     # -----------------------------
#     # Place Orders
#     # -----------------------------
#     place_btn = widgets.Button(description="Place All Orders", button_style="success")
#     def place_orders(b):
#         with output:
#             clear_output()
#             if not orders:
#                 print("⚠️ No orders to place!")
#                 return
#             print("🚀 Placing all orders...\n")
#             for order in orders:
#                 try:
#                     order_id = kite.place_order(
#                         variety=order["variety"],
#                         exchange=order["exchange"],
#                         tradingsymbol=order["tradingsymbol"],
#                         transaction_type=order["transaction_type"],
#                         quantity=order["quantity"],
#                         order_type=order["order_type"],
#                         product=order["product"]
#                     )
#                     display(HTML(f"<pre style='color: green; font-weight: bold'>✔ Order placed: {order['tradingsymbol']} → Order ID: {order_id}</pre>"))
#                 except Exception as e:
#                     display(HTML(f"<pre style='color: red; font-weight: bold'>❌ Error placing {order['tradingsymbol']}: {e}</pre>"))
#     place_btn.on_click(place_orders)

#     # -----------------------------
#     # Display FUTURES and OPTIONS side by side
#     # -----------------------------
#     fut_box = widgets.VBox([
#         widgets.HTML("<h4>FUTURES</h4>"),
#         fut_widget["tradingsymbol"],
#         fut_widget["side_box"],  # separated from symbol by VBox margin
#         fut_widget["qty_box"],
#         fut_widget["order_type"],
#         fut_widget["product"],
#         fut_widget["variety"],
#         fut_widget["add_btn"]
#     ], layout=widgets.Layout(min_width='320px'))

#     opt_box = widgets.VBox([
#         widgets.HTML("<h4>OPTIONS</h4>"),
#         opt_widget["tradingsymbol"],
#         opt_widget["side_box"],  # separated from symbol by VBox margin
#         opt_widget["qty_box"],
#         opt_widget["order_type"],
#         opt_widget["product"],
#         opt_widget["variety"],
#         opt_widget["add_btn"]
#     ], layout=widgets.Layout(min_width='320px'))

#     container = widgets.HBox(
#         [fut_box, opt_box],
#         layout=widgets.Layout(
#             justify_content='space-between',
#             width='100%',
#             padding='0 50px'
#         )
#     )

#     display(container)
#     display(widgets.HBox([place_btn, clear_btn], layout=widgets.Layout(margin='20px 0 0 0')))
#     display(output)


In [ ]:
# import pandas as pd
# import ipywidgets as widgets
# from IPython.display import display, clear_output, HTML

# def make_order_basket_check_margins(fut_symbols, option_symbols, kite):
#     orders = []
#     qty_options = [75, 150, 225, 300, 375, 450]
#     kite_orange = '#f57c00'
#     output = widgets.Output()
    
#     # -----------------------------
#     # Quantity Buttons
#     # -----------------------------
#     def create_qty_buttons():
#         selected_qty = qty_options[0]
#         buttons = []
#         for q in qty_options:
#             btn = widgets.Button(description=str(q),
#                                  layout=widgets.Layout(width='60px', margin='0 8px 0 0'))
#             buttons.append(btn)

#         qty_out = widgets.Output()
#         qty_box = widgets.VBox([widgets.HBox(buttons + [qty_out])],
#                                layout=widgets.Layout(margin='0 0 10px 0'))

#         def update_styles(clicked_btn):
#             nonlocal selected_qty
#             selected_qty = int(clicked_btn.description)
#             for b in buttons:
#                 b.style.button_color = kite_orange if b == clicked_btn else None
#             with qty_out:
#                 clear_output()

#         for b in buttons:
#             b.on_click(update_styles)

#         update_styles(buttons[0])
#         return qty_box, lambda: selected_qty

#     # -----------------------------
#     # Buy/Sell Buttons
#     # -----------------------------
#     def create_side_buttons(default='BUY'):
#         selected_side = default
#         buy_btn = widgets.Button(description="BUY",
#                                  layout=widgets.Layout(width='60px', margin='0 8px 0 0'))
#         sell_btn = widgets.Button(description="SELL",
#                                   layout=widgets.Layout(width='60px', margin='0 8px 0 0'))

#         side_box = widgets.VBox([widgets.HBox([buy_btn, sell_btn])],
#                                 layout=widgets.Layout(margin='0 0 10px 0'))

#         def update_styles(btn_clicked):
#             nonlocal selected_side
#             selected_side = btn_clicked.description
#             buy_btn.style.button_color = '#2196f3' if selected_side == 'BUY' else '#ccc'
#             sell_btn.style.button_color = 'red' if selected_side == 'SELL' else '#ccc'

#         buy_btn.on_click(update_styles)
#         sell_btn.on_click(update_styles)
#         update_styles(widgets.Button(description=selected_side))
#         return side_box, lambda: selected_side

#     # -----------------------------
#     # Order Widget Set
#     # -----------------------------
#     def create_order_widget(symbols_list, default_buy=True):
#         tradingsymbol = widgets.Dropdown(options=symbols_list, description="Symbol:",
#                                          layout=widgets.Layout(margin='0 0 10px 0'))
#         side_box, get_side = create_side_buttons('BUY' if default_buy else 'SELL')
#         qty_box, get_qty = create_qty_buttons()
#         order_type = widgets.Dropdown(options=["MARKET", "LIMIT"], value="MARKET",
#                                       description="Type:", layout=widgets.Layout(margin='0 0 10px 0'))
#         product = widgets.Dropdown(options=["MIS", "CNC", "NRML"], value="MIS",
#                                    description="Product:", layout=widgets.Layout(margin='0 0 10px 0'))
#         variety = widgets.Dropdown(options=["regular", "amo"], value="amo",
#                                    description="Variety:", layout=widgets.Layout(margin='0 0 10px 0'))
#         add_btn = widgets.Button(description="Add Order", button_style="primary",
#                                  layout=widgets.Layout(margin='0 0 20px 0'))

#         return {
#             "tradingsymbol": tradingsymbol,
#             "get_side": get_side,
#             "side_box": side_box,
#             "qty_box": qty_box,
#             "get_qty": get_qty,
#             "order_type": order_type,
#             "product": product,
#             "variety": variety,
#             "add_btn": add_btn
#         }

#     fut_widget = create_order_widget(fut_symbols, default_buy=True)
#     opt_widget = create_order_widget(option_symbols, default_buy=True)

#     # -----------------------------
#     # Add Order Logic
#     # -----------------------------
#     def add_order_widget(widget):
#         sym = widget["tradingsymbol"].value
#         txn_type = widget["get_side"]()
#         qty = widget["get_qty"]()

#         if sym.endswith(("CE", "PE")):
#             txn_type = "BUY"

#         order = {
#             "tradingsymbol": sym,
#             "exchange": "NFO",
#             "transaction_type": txn_type,
#             "quantity": qty,
#             "order_type": widget["order_type"].value,
#             "product": widget["product"].value,
#             "variety": widget["variety"].value
#         }

#         orders.append(order)

#         with output:
#             clear_output()
#             display(HTML("<span style='color: orange; font-weight: bold;'>🛒 Order Basket:</span>"))
#             for o in orders:
#                 side_color = "green" if o["transaction_type"] == "BUY" else "red"
#                 box_style = (
#                     "display: inline-block; padding: 5px 10px; margin: 2px 4px; "
#                     "border-radius: 5px; font-family: monospace; font-weight: bold; color: white;"
#                 )
#                 symbol_box = f"<span style='{box_style} background-color: #2196f3;'>{o['tradingsymbol']}</span>"
#                 qty_box = f"<span style='{box_style} background-color: #2196f3;'>{o['quantity']}</span>"
#                 side_box_html = f"<span style='{box_style} background-color: {side_color};'>{o['transaction_type']}</span>"
#                 type_box = f"<span style='{box_style} background-color: #2196f3;'>{o['order_type']}</span>"
#                 product_box = f"<span style='{box_style} background-color: #2196f3;'>{o['product']}</span>"
#                 variety_box = f"<span style='{box_style} background-color: #2196f3;'>{o['variety']}</span>"
#                 display(HTML(symbol_box + qty_box + side_box_html + type_box + product_box + variety_box))

#     fut_widget["add_btn"].on_click(lambda b: add_order_widget(fut_widget))
#     opt_widget["add_btn"].on_click(lambda b: add_order_widget(opt_widget))

#     # -----------------------------
#     # Clear Orders
#     # -----------------------------
#     clear_btn = widgets.Button(description="Clear Orders", button_style="danger")
#     def clear_orders(b):
#         with output:
#             clear_output()
#             orders.clear()
#             print("🗑️ All orders cleared.")
#     clear_btn.on_click(clear_orders)

#     # -----------------------------
#     # Place Orders
#     # -----------------------------
#     place_btn = widgets.Button(description="Place All Orders", button_style="success")
#     def place_orders(b):
#         with output:
#             clear_output()
#             if not orders:
#                 print("⚠️ No orders to place!")
#                 return

#             # Check margin before placing
#             margin_ok = check_margin_required()
#             if not margin_ok:
#                 print("⚠️ Insufficient funds to place these orders!")
#                 return

#             print("🚀 Placing all orders...\n")
#             for order in orders:
#                 try:
#                     order_id = kite.place_order(
#                         variety=order["variety"],
#                         exchange=order["exchange"],
#                         tradingsymbol=order["tradingsymbol"],
#                         transaction_type=order["transaction_type"],
#                         quantity=order["quantity"],
#                         order_type=order["order_type"],
#                         product=order["product"]
#                     )
#                     display(HTML(f"<pre style='color: green; font-weight: bold'>✔ Order placed: {order['tradingsymbol']} → Order ID: {order_id}</pre>"))
#                 except Exception as e:
#                     display(HTML(f"<pre style='color: red; font-weight: bold'>❌ Error placing {order['tradingsymbol']}: {e}</pre>"))
#     place_btn.on_click(place_orders)

#     # -----------------------------
#     # Check Margin
#     # -----------------------------
#     margin_btn = widgets.Button(description="Check Margin", button_style="info")
#     def check_margin_required(b=None):
#         if not orders:
#             with output:
#                 clear_output()
#                 print("⚠️ No orders to check margin for!")
#             return False

#         # Convert orders to NSE format for margin calculation
#         margin_orders = []
#         for o in orders:
#             ex = "NSE" if not o["tradingsymbol"].endswith(("CE","PE")) else "NFO"
#             margin_orders.append({
#                 "exchange": ex,
#                 "tradingsymbol": o["tradingsymbol"],
#                 "transaction_type": o["transaction_type"],
#                 "quantity": o["quantity"],
#                 "product": o["product"],
#                 "order_type": o["order_type"]
#             })

#         try:
#             margins = kite.margins()
#             margin_info = kite.basket_order_margins(margin_orders, consider_positions=True, mode="compact")
#             available_balance = margins['equity']['net']
#             df = pd.json_normalize(margin_info['orders'], sep='_')
#             total_required = df['total'].sum()

#             with output:
#                 clear_output()
#                 print(f"💰 Available Balance: ₹{available_balance:,.2f}")
#                 print(f"📊 Margin Required: ₹{total_required:,.2f}")
#                 if total_required > available_balance:
#                     print("⚠️ Insufficient funds to place these orders!")
#                     return False
#                 else:
#                     print("✅ Sufficient funds available.")
#                     return True
#         except Exception as e:
#             with output:
#                 clear_output()
#                 print(f"❌ Error checking margin: {e}")
#             return False

#     margin_btn.on_click(check_margin_required)

#     # -----------------------------
#     # Layout FUTURES and OPTIONS side by side
#     # -----------------------------
#     fut_box = widgets.VBox([
#         widgets.HTML("<h4>FUTURES</h4>"),
#         fut_widget["tradingsymbol"],
#         fut_widget["side_box"],
#         fut_widget["qty_box"],
#         fut_widget["order_type"],
#         fut_widget["product"],
#         fut_widget["variety"],
#         fut_widget["add_btn"]
#     ], layout=widgets.Layout(min_width='320px'))

#     opt_box = widgets.VBox([
#         widgets.HTML("<h4>OPTIONS</h4>"),
#         opt_widget["tradingsymbol"],
#         opt_widget["side_box"],
#         opt_widget["qty_box"],
#         opt_widget["order_type"],
#         opt_widget["product"],
#         opt_widget["variety"],
#         opt_widget["add_btn"]
#     ], layout=widgets.Layout(min_width='320px'))

#     container = widgets.HBox(
#         [fut_box, opt_box],
#         layout=widgets.Layout(
#             justify_content='space-between',
#             width='100%',
#             padding='0 50px'
#         )
#     )

#     display(container)
#     display(widgets.HBox([place_btn, clear_btn, margin_btn], layout=widgets.Layout(margin='20px 0 0 0')))
#     display(output)
